![uc3m](img/uc3m.jpg)

# Standard formulations and basic solutions

<a href="http://www.est.uc3m.es/nogales" target="_blank">Javier Nogales</a>

Consider the Airline Revenue Management example (from slides)

![](img/RM_matrix.png)

## Solving linear systems in Python

In the example, we are going to obtain two more feasible basic solutions, S9 and S10 

In [3]:
#      1 1 1 0 0
#A = ( 1 0 0 1 0 )
#      0 1 0 0 1
    
import numpy as np

# Basic solution corresponding to (x1, x4, x5)
# 3 Dimentions
B = np.array([[1, 0, 0], 
              [0, 1, 0], 
              [0, 0, 1]])

b = np.array([150, 75, 125])

x_B = np.linalg.solve(B,b) #Solve B * X = b
print(x_B)

# Basic solution corresponding to (x2, x4, x5)
B = np.array([[1, 0, 1], 
              [0, 1, 0], 
              [1, 0, 0]])
b = np.array([150, 75, 125])
x_B = np.linalg.solve(B,b)
print(x_B)

# When all values positive => CORNER

[150.  75. 125.]
[125.  75.  25.]


Are they extreme points (feasible basic solutions)?

## Standard Formulation with Pyomo

We can use Pyomo to model problems with matrix format using Numpy arrays to enter the data

The indices are represented by the Python range() statement.


In [ ]:
from pyomo.environ import *
import numpy as np

A = np.array([[1,1,1,0,0],
              [1,0,0,1,0],
              [0,1,0,0,1]])

b = np.array([150,75,125])

c = np.array([-400,-150,0,0,0])

# set of row indices
I = range(len(A))

# set of column indices
J = range(len(A.T))

# create a model instance
model = ConcreteModel()

# all the variables (original plus slacks)
model.x = Var(J, domain=NonNegativeReals)

# the objective
model.objective = Objective(expr = sum(c[j]*model.x[j] for j in J), sense=minimize)

# the constraints (added row by row)
model.constraints = ConstraintList()
for i in I:
    model.constraints.add(sum(A[i,j]*model.x[j] for j in J) == b[i])

# the solver
solver = SolverFactory('glpk')

solver.solve(model)

# print solutions (in python indexes start from 0)
for j in J:
    print("x[", j, "] =", model.x[j].value)

model.constraints.display()

model.objective()

But remember in Pyomo there is no need to enter the problem in its standard formulation, we just need to enter the problem using the original formulation, which is easier and more natural to understand:

In [ ]:
model = ConcreteModel()

model.x = Var([1,2], domain=NonNegativeReals)
model.obj = Objective(expr = 400*model.x[1]+150*model.x[2],sense = maximize)
model.cons1 = Constraint(expr = model.x[1]+model.x[2]<=150)
model.cons2 = Constraint(expr = model.x[1]<=75)
model.cons3 = Constraint(expr = model.x[2]<=125)

# obtain the solution 
Solver = SolverFactory('glpk')
Results = Solver.solve(model)
model.x.display()

# The Simplex Algorithm using SciPy

With SciPy, we can optimize a linear objective function subject to linear equality and inequality constraints of the following form:

![](img/SciPy.png)

In [ ]:
import numpy as np

from scipy.optimize import linprog

A = np.array([[1,1,1,0,0],
              [1,0,0,1,0],
              [0,1,0,0,1]])

b = np.array([150,75,125])

c = np.array([-400,-150,0,0,0])

sol = linprog(c, A_eq=A, b_eq=b, bounds=(0, None))

print('Optimal value:', sol.fun, '\nX:', sol.x)



Optimal solution: $(x_1,x_2)=(75,75)$

Optimal value: 41250